## Embedding Training for the Concept Space
1. Each Concept has an embedding
2. Use a matrix to represent all the relations.

3. Head concept and Tail concept --> Fully-connected layer --> hidden state --> relation context --> close to target relations

4. map text spans into points in the concept space (wikipedia/dictionary descriptions)
5. concept relations --> (text: OpenIE extracted relations), map Head tail concepts into the knowledge space, than force in text relations to predict the relations in the knowledge  space
When training , how to deal with multi-hop reasoning?



## Installation:
1. Download BabelNet Index
2. Download WordNet3.0
3. Download BabelNet Java API
4. Configure babelnet.var.properties (path/to/extracted BabelNet Index)
   Configure jlt.var.properties (path/to/wordnet prefix)
5. In eclipse Maven, configure pom.xml
   load BabelNet java API as external jars

## Text to Concept
1. Input a text span, get an vector in the concept space
2. Find top K concepts in the concept space, closest to this transformed vector
3. Sample H hops from each of the K top concepts
4. generate concept graphs

## Reasoning in the Concept Space
- For premise-assertion pairs, build concept graph for the two text spans respectively, using all the text spans in each (could be OpenIE results, or fixed window spans for the Linking concept transformed from the text)
  1. pretrained concept vectors and one-hot edge types as node and edge features, encode using graph_nets
  2. get the global hidden state of both graph seperatedly as the background context
  3. max the relation feature vector (R) of the merged graph
  4. max the node feature vectors (N) of the merged graph
  5. minuse premise and assertion grpah from the merged graph for global hidden state, node/edge level
  6, concatenate the difference vectors (3) with seperate graphs and go through a MLP to predict
- For single sentence, force sentence relations to be closer to the edge hidden states of the concepts in the concept space.
  1. pretrained concept vectors and one-hot edge types as node and edge features, encode using graph_nets
  2. get the global hidden states and relation hidden state (max/averageed)
  3. the hidden state is forced to be closer to the text OPenIE (or whole sentence) relation


## Datasets
### [AI2 Reasoning Challenge (ARC)](https://s3-us-west-2.amazonaws.com/ai2-website/data/ARC-V1-Feb2018.zip)

Think you have solved question answering? Try the AI2 Reasoning Challenge (ARC)! The ARC dataset contains 7,787 genuine grade-school level, multiple-choice science questions, assembled to encourage research in advanced question-answering. The dataset is partitioned into a Challenge Set and an Easy Set, where the former contains only questions answered incorrectly by both a retrieval-based algorithm and a word co-occurrence algorithm. The questions are partitioned as follows:

  - Challenge Train: 1,119
  - Challenge Dev: 299
  - Challenge Test: 1,172
  - Easy Train: 2,251
  - Easy Dev: 570
  - Easy Test: 2,376
  

### [SciTail Dataset](http://data.allenai.org.s3.amazonaws.com/downloads/SciTailV1.1.zip)

The SciTail dataset is an entailment dataset created from multiple-choice science exams and web sentences. Each question and the correct answer choice are converted into an assertive statement to form the hypothesis. We use information retrieval to obtain relevant text from a large text corpus of web sentences, and use these sentences as a premise P. We crowdsource the annotation of such premise-hypothesis pair as supports (entails) or not (neutral), in order to create the SciTail dataset.The dataset contains 27,026 examples with 10,101 examples with entails label and 16,925 examples with neutral label.
A limitation of mainstream entailment datasets is that they have been constructed in isolation from any end task. Moreover, in several cases, either the hypothesis or the premise has been synthesized specifically for creating the entailment dataset. Both the premise and the hypothesis in SciTail were authored independently of each other and independent of the entailment task. As a result, linguistic variations in the dataset are not limited by the coverage of rules or the creativity of crowd-workers. Further, unfiltered web sentences, which are used to create the premises, tend to be highly diverse in various aspects (length, complexity, being well-formed for a parser, etc.), adding to the linguistic challenge. Refer to our paper on SciTail, A Textual Entailment Dataset from Science Question Answering for additional information.

QUESTION:
Which of the following best explains how stems transport water to other parts of the plant?

- (A) through a chemical called chlorophyll.
- (B) by using photosynthesis.
- (C) through a system of tubes.
- (D) by converting water to food.

HYPOTHESIS FROM QUESTION + ANSWER CANDIDATE (C):
Stems transport water to other parts of the plant through a system of tubes.

SUPPORTING PREMISE (ENTAILS):
Water and other materials necessary for biological activity in trees are transported throughout the stem and branches in thin, hollow tubes in the xylem, or wood tissue.

NON-SUPPORTING PREMISE (NEUTRAL):
Cut plant stems and insert stem into tubing while stem is submerged in a pan of water.

### [AI2 Science Questions v2.1 (October 2017)](https://s3-us-west-2.amazonaws.com/ai2-website/data/AI2-ScienceQuestions-V2-1-October2017.zip)

textbook_image

The AI2 Science Questions dataset consists of questions used in student assessments in the United States across elementary and middle school grade levels. Each question is 4-way multiple choice format and may or may not include a diagram element.

- Elementary School Without Diagrams v2: 1,288 questions
- Elementary School With Diagrams v2: 1,183 questions
- Middle School Without Diagrams v2: 1,409 questions
- Middle School With Diagrams v2: 1,179 questions
- The question train/dev/test sets within these categories are named using the following acronyms for brevity:

- NDMC: Non-Diagram Multiple Choice
- DMC: Diagram Multiple Choice

### [SciQ Dataset](https://s3-us-west-2.amazonaws.com/ai2-website/data/SciQ.zip)

This dataset contains 13679 crowdsourced science exam questions about Physics, Chemistry and Biology, among others. The questions are in multiple-choice format with 4 answer options each. For the majority of the questions, an additional paragraph with supporting evidence for the correct answer is provided.


### [Question Answering Challenge Targeting Commonsense Knowledge ](https://www.dropbox.com/sh/r4187lyw5ojd5x4/AAALXhvhhzfZa-UjaycBUb-Da)

Question Answering Challenge Targeting Commonsense Knowledge 


CommonsenseQA is a new multiple-choice question answering dataset that requires different types of commonsense knowledge to predict the correct answers . It contains 9,500 questions with one correct answer and two distractor answers.  The dataset is provided in two major training/validation/testing set splits: "Random split" which is the main evaluation split, and "Question token split", see paper for details.

In [ ]:
# Read Data
["(Google)"," (dictionary.com)"]
empty_markers = ["(appropriate sense not in WordNet)","(absent in WordNet)"]
# if it is concept, get all concept synsets, else, get named entiry synsets
# wn:02082791n == 102082791
# http://localhost:8779/sense/?term=aardvark

1. Read in the vocab file
   fix the POS, category
   1.1 fixed target learning
      for each vocab, only find one concept/entity that best matches it.
   1.2 multi-sense support
      for each vocab, find all concepts or the very entity that matche(s) it.
   output is concept --> glosses

# Ensure that BabelNet api server is running at port 8779

In [2]:
import requests
import pandas as pd
import os

url_base = "http://localhost:8779/sense/?term={}"
bn_pos_notations = ['NOUN','ADJ','VERB']
db_pos_notations = ['n','adj','v']
interested_info_bn = ['grosses', 'bnid','wnids','categories','examples','pos','type']
interested_info_db1 = ["is_entity","entity_type"]
interested_info_db2 = ['POS','Category','Canonical Name','WN30 Synset','WordNet gloss']


In [3]:
db_path = '/home/chen/projects/text2concept/concept_space/data/aristo-tuple-kb-v5-mar2017/aristo-tuple-kb-v5-mar2017'
entity_dict_path = os.path.join(db_path,'entities.tsv')
vocab_path = os.path.join(db_path, 'Domain Vocabulary - mar2016.tsv')
# df_db = pd.read_csv(vocab_path, sep='\t', chunksize=10*6)
df_db = pd.read_csv(vocab_path, sep='\t')
df_entities = pd.read_csv(entity_dict_path, sep='\t')

In [4]:
import shutil
import pickle
from sqlalchemy import create_engine
import json
import tqdm

aligned_data_path = "aligned_data"
database_filename = "VocabDB.db"
table_name = "t_vocab"

# delete and reconstruct the db if already existing one
if aligned_data_path in os.listdir():
    shutil.rmtree(aligned_data_path, )
os.mkdir(aligned_data_path)

engine = create_engine('sqlite:///{}'.format(os.path.join(aligned_data_path,database_filename)), echo=False)

# check pos matching
mapping = {"n":"NOUN","v":"VERB", "adj":"ADJ"}
def pos_match(bn_pos, db_pos):
    if db_pos in mapping:
        if mapping[db_pos] == bn_pos:
            return True
        else:
            return False
    return True

In [5]:
is_first = True

for i in df_db.index:
    df_record = df_db.loc[i]
    df_record = pd.DataFrame([df_record.values], columns=df_record.index)
    print(df_record['Word'].item())
    word = "{}".format(df_record['Word'].item()).strip()
#     db_desc = df_record['WordNet gloss'].item()
    try:
        resp = requests.get(url_base.format(word))
        info = resp.json()

        # check type of the term: NER or Concept
        bn_vocab_type, db_ner_type = "Concept", ""
        df_ner = df_entities.query('Word == "{}"'.format(word))
        if df_ner.shape[0] > 0:
            db_ner_type = df_ner['Type'].item()

        # filter irrelevant synsets
        bn_synsets = []
        for synset in info:
            if pos_match(synset['pos'], df_record['POS'].item()):
                if 'senses' in synset:
                    synset.pop('senses')  # Remove sense records
                synset['relations'] = [rel for rel in synset['relations'] if 'semantically_related_form' not in rel]
                bn_synsets.append(synset.copy())
                
        # update data and persistence
        bn_synsets = json.dumps(bn_synsets)
        df_record['bn_vocab_type'] = bn_vocab_type
        df_record['db_ner_type'] = db_ner_type
        df_record['bn_synsets'] = bn_synsets
        df_record.to_sql(table_name, con=engine, if_exists='append', index=False)
        print("Finished {}th term {}.".format(i, word))
    except Exception as e:
        print(e)
        print("!Failed  {}!!".format(df_record['Word']))
        continue
        
            
        

aardvark
Finished 0th term aardvark.
aardwolf
Finished 1th term aardwolf.
abandon
Finished 2th term abandon.
abbey
Finished 3th term abbey.
abdomen
Finished 4th term abdomen.
abdominal
Finished 5th term abdominal.
ability
Finished 6th term ability.
able
Finished 7th term able.
abnormal
Finished 8th term abnormal.
abnormality
Finished 9th term abnormality.
abnormally
Finished 10th term abnormally.
aboard
Finished 11th term aboard.
abound
Finished 12th term abound.
about
Finished 13th term about.
above
Finished 14th term above.
abracadabra
Finished 15th term abracadabra.
abrasion
Finished 16th term abrasion.
abroad
Finished 17th term abroad.
abrupt
Finished 18th term abrupt.
abruptly
Finished 19th term abruptly.
absence
Finished 20th term absence.
absent
Finished 21th term absent.
absolute
Finished 22th term absolute.
absolutely
Finished 23th term absolutely.
absorb
Finished 24th term absorb.
absorbent
Finished 25th term absorbent.
absorption
Finished 26th term absorption.
absorptive
Fin

Finished 213th term aircraft.
airlift
Finished 214th term airlift.
airplane
Finished 215th term airplane.
airport
Finished 216th term airport.
airtight
Finished 217th term airtight.
airway
Finished 218th term airway.
Alabama
Finished 219th term Alabama.
alanine
Finished 220th term alanine.
alarm
Finished 221th term alarm.
alarm
Finished 222th term alarm.
alarm
Finished 223th term alarm.
Alaska
Finished 224th term Alaska.
Albania
Finished 225th term Albania.
Albany
Finished 226th term Albany.
albatross
Finished 227th term albatross.
Alberta
Finished 228th term Alberta.
albinism
Finished 229th term albinism.
albino
Finished 230th term albino.
album
Finished 231th term album.
albumen
Finished 232th term albumen.
alcohol
Finished 233th term alcohol.
alcoholic
Finished 234th term alcoholic.
alder
Finished 235th term alder.
alert
Finished 236th term alert.
alga
Finished 237th term alga.
algae
Finished 238th term algae.
algal
Finished 239th term algal.
Algeria
Finished 240th term Algeria.
Alg

Finished 432th term ape.
aperture
Finished 433th term aperture.
aphelion
Finished 434th term aphelion.
aphid
Finished 435th term aphid.
aphotic
Finished 436th term aphotic.
apologise
Finished 437th term apologise.
apologize
Finished 438th term apologize.
Appalachian
Finished 439th term Appalachian.
apparatus
Finished 440th term apparatus.
apparent
Finished 441th term apparent.
apparently
Finished 442th term apparently.
appear
Finished 443th term appear.
appearance
Finished 444th term appearance.
append
Finished 445th term append.
appendage
Finished 446th term appendage.
appendix
Finished 447th term appendix.
appetite
Finished 448th term appetite.
appetizing
Finished 449th term appetizing.
applause
Finished 450th term applause.
apple
Finished 451th term apple.
appliance
Finished 452th term appliance.
application
Finished 453th term application.
apply
Finished 454th term apply.
appreciation
Finished 455th term appreciation.
apprehend
Finished 456th term apprehend.
approach
Finished 457th

Finished 648th term autoimmune.
automatic
Finished 649th term automatic.
automatically
Finished 650th term automatically.
automaton
Finished 651th term automaton.
automobile
Finished 652th term automobile.
automotive
Finished 653th term automotive.
autonomic
Finished 654th term autonomic.
autonomous
Finished 655th term autonomous.
autosomal
Finished 656th term autosomal.
autosome
Finished 657th term autosome.
autotroph
Finished 658th term autotroph.
autumn
Finished 659th term autumn.
autumnal
Finished 660th term autumnal.
auxin
Finished 661th term auxin.
availability
Finished 662th term availability.
available
Finished 663th term available.
avalanche
Finished 664th term avalanche.
average
Finished 665th term average.
aversion
Finished 666th term aversion.
avocado
Finished 667th term avocado.
avoid
Finished 668th term avoid.
awake
Finished 669th term awake.
awaken
Finished 670th term awaken.
award
Finished 671th term award.
award
Finished 672th term award.
aware
Finished 673th term awar

Finished 879th term Belgium.
belief
Finished 880th term belief.
believe
Finished 881th term believe.
Belize
Finished 882th term Belize.
bell
Finished 883th term bell.
belladonna
Finished 884th term belladonna.
belle
Finished 885th term belle.
bellow
Finished 886th term bellow.
belly
Finished 887th term belly.
belong
Finished 888th term belong.
beloved
Finished 889th term beloved.
below
Finished 890th term below.
belt
Finished 891th term belt.
belt
Finished 892th term belt.
beluga
Finished 893th term beluga.
bench
Finished 894th term bench.
benchmark
Finished 895th term benchmark.
bend
Finished 896th term bend.
beneath
Finished 897th term beneath.
beneficial
Finished 898th term beneficial.
benefit
Finished 899th term benefit.
benevolent
Finished 900th term benevolent.
benign
Finished 901th term benign.
Benin
Finished 902th term Benin.
bent
Finished 903th term bent.
benzene
Finished 904th term benzene.
beret
Finished 905th term beret.
Berkeley
Finished 906th term Berkeley.
Berkshire
Fini

...

Finished 10587th term toxin.
toy
Finished 10588th term toy.
trace
Finished 10589th term trace.
trachea
Finished 10590th term trachea.
tracheophyte
Finished 10591th term tracheophyte.
track
Finished 10592th term track.
tract
Finished 10593th term tract.
traction
Finished 10594th term traction.
tractor
Finished 10595th term tractor.
trade
Finished 10596th term trade.
trademark
Finished 10597th term trademark.
trader
Finished 10598th term trader.
tradition
Finished 10599th term tradition.
traditional
Finished 10600th term traditional.
traditionally
Finished 10601th term traditionally.
traffic
Finished 10602th term traffic.
trail
Finished 10603th term trail.
trail
Finished 10604th term trail.
trail
Finished 10605th term trail.
trailer
Finished 10606th term trailer.
train
Finished 10607th term train.
trainer
Finished 10608th term trainer.
trait
Finished 10609th term trait.
tram
Finished 10610th term tram.
tramp
Finished 10611th term tramp.
trampoline
Finished 10612th term trampoline.
trance

Finished 10797th term typical.
typically
Finished 10798th term typically.
tyrannosaurus
Finished 10799th term tyrannosaurus.
tyre
Finished 10800th term tyre.
tyrosine
Finished 10801th term tyrosine.
uakari
Finished 10802th term uakari.
Uganda
Finished 10803th term Uganda.
ugly
Finished 10804th term ugly.
Ukraine
Finished 10805th term Ukraine.
ulcer
Finished 10806th term ulcer.
ultimate
Finished 10807th term ultimate.
ultimately
Finished 10808th term ultimately.
ultrasonic
Finished 10809th term ultrasonic.
ultraviolet
Finished 10810th term ultraviolet.
umbilical
Finished 10811th term umbilical.
umbrella
Finished 10812th term umbrella.
unable
Finished 10813th term unable.
unacceptable
Finished 10814th term unacceptable.
unaffected
Finished 10815th term unaffected.
unaided
Finished 10816th term unaided.
unaltered
Finished 10817th term unaltered.
unambiguous
Finished 10818th term unambiguous.
unavailable
Finished 10819th term unavailable.
unaware
Finished 10820th term unaware.
unbalance
Fi

Finished 10995th term ut.
Utah
Finished 10996th term Utah.
utensil
Finished 10997th term utensil.
uterus
Finished 10998th term uterus.
utilise
Finished 10999th term utilise.
utility
Finished 11000th term utility.
utilize
Finished 11001th term utilize.
utmost
Finished 11002th term utmost.
utter
Finished 11003th term utter.
utterly
Finished 11004th term utterly.
Uzbekistan
Finished 11005th term Uzbekistan.
vacate
Finished 11006th term vacate.
vacation
Finished 11007th term vacation.
vaccine
Finished 11008th term vaccine.
vacuole
Finished 11009th term vacuole.
vacuum
Finished 11010th term vacuum.
vagina
Finished 11011th term vagina.
vaginal
Finished 11012th term vaginal.
vagrant
Finished 11013th term vagrant.
vagrant
Finished 11014th term vagrant.
vain
Finished 11015th term vain.
valence
Finished 11016th term valence.
valid
Finished 11017th term valid.
valley
Finished 11018th term valley.
valuable
Finished 11019th term valuable.
value
Finished 11020th term value.
valve
Finished 11021th te

Finished 11209th term ward.
warden
Finished 11210th term warden.
wardrobe
Finished 11211th term wardrobe.
warm
Finished 11212th term warm.
warmth
Finished 11213th term warmth.
warn
Finished 11214th term warn.
warning
Finished 11215th term warning.
warp
Finished 11216th term warp.
warrant
Finished 11217th term warrant.
warren
Finished 11218th term warren.
warrior
Finished 11219th term warrior.
wart
Finished 11220th term wart.
warty
Finished 11221th term warty.
wash
Finished 11222th term wash.
washbasin
Finished 11223th term washbasin.
washer
Finished 11224th term washer.
washing-up
Finished 11225th term washing-up.
Washington
Finished 11226th term Washington.
wasp
Finished 11227th term wasp.
waste
Finished 11228th term waste.
wastewater
Finished 11229th term wastewater.
watch
Finished 11230th term watch.
water
Finished 11231th term water.
waters
Finished 11232th term waters.
waterbuck
Finished 11233th term waterbuck.
watercress
Finished 11234th term watercress.
waterfall
Finished 11235t

Finished 11427th term woodwork.
woody
Finished 11428th term woody.
woof
Finished 11429th term woof.
wool
Finished 11430th term wool.
woolly
Finished 11431th term woolly.
word
Finished 11432th term word.
wordnet
Finished 11433th term wordnet.
work
Finished 11434th term work.
workbench
Finished 11435th term workbench.
worker
Finished 11436th term worker.
workman
Finished 11437th term workman.
workout
Finished 11438th term workout.
workplace
Finished 11439th term workplace.
workshop
Finished 11440th term workshop.
workshop
Finished 11441th term workshop.
world
Finished 11442th term world.
worldwide
Finished 11443th term worldwide.
worm
Finished 11444th term worm.
wormlike
Finished 11445th term wormlike.
worrier
Finished 11446th term worrier.
worry
Finished 11447th term worry.
worship
Finished 11448th term worship.
worship
Finished 11449th term worship.
wort
Finished 11450th term wort.
worth
Finished 11451th term worth.
worthwhile
Finished 11452th term worthwhile.
worthy
Finished 11453th t

In [6]:
import spacy
# Load English tokenizer, tagger, parser, NER and word vectors
nlp = spacy.load('en_core_web_lg')

In [7]:
def get_cleaned_categories(cat_list):
    cats = []
    for cat in cat_list:
        cat = " ".join(cat[cat.rfind(":")+1:].split('_'))
        cats.append(cat)
    return cats

In [12]:
# prepare dbs 
import pickle
from sqlalchemy import create_engine
import json
import numpy as np

aligned_data_path = "aligned_data"
database_filename = "VocabDB.db"
cleaned_filename = "FlatternedDB.db"
table_name = "t_vocab"
chunksize = 100

raw_db_path = os.path.join(aligned_data_path, database_filename)
new_db_path = os.path.join(aligned_data_path, cleaned_filename)
if os.path.exists(new_db_path):
    os.remove(new_db_path)

raw_engine = create_engine("sqlite:///{}".format(raw_db_path))    
new_engine = create_engine('sqlite:///{}'.format(new_db_path), echo=False)

# new table columns to write
columns = ['Word','gloss','type']


In [13]:
# clean db data and generate description concept pairs
r_con = raw_engine.connect()
r_results = r_con.execute('select * from {}'.format(table_name))
for idx, row in enumerate(r_results):
    
    spacy_gloss, spacy_category = "", ""
    
    word = row['Word']
    gloss = row['WordNet gloss']
    example = row['WordNet example usage']
    print("Processing {}th word {}.".format(idx, word))
    
    # self mapping should always be true
    pd.DataFrame([[word, word, 'self']], columns=columns).to_sql(table_name, con=new_engine, if_exists='append', index=False)
    
    if not pd.isnull(gloss) and len(gloss) > 3:
        pd.DataFrame([[word, gloss, 'db_gloss']], columns=columns).to_sql(table_name, con=new_engine, if_exists='append', index=False)
        spacy_gloss = nlp(gloss)
    
    if not pd.isnull(example) and len(example) > 3:
        pd.DataFrame([[word, example, 'db_example']], columns=columns).to_sql(table_name, con=new_engine, if_exists='append', index=False)
        
    category = row["Category"] if not row["Category"] is None else ""
    category += " {}".format(" ".join(row['db_ner_type'].split('_')[:-1])) if not row['db_ner_type'] is None else ""
    
    if category:
        spacy_category = nlp(category)
        
    # align by id or align by category or align by definition
    # find the most matching concept in the babelnet knowledge graph
    matched_synset = None
    highest_score = -1
    for synset in json.loads(row['bn_synsets']):
        cat_sim, gloss_sim = 0, 0
        bn_cats = get_cleaned_categories(synset['categories'])
        bn_glosses = synset['glosses']
        if spacy_category != "" and bn_cats:
            cat_sim = max(cat_sim, np.max([nlp(cat).similarity(spacy_category) for cat in bn_cats]))
        if spacy_gloss != "" and bn_glosses:
            gloss_sim = max(gloss_sim, np.max([nlp(glz).similarity(spacy_gloss) for glz in bn_glosses]))
        cur_sim = cat_sim + gloss_sim
        highest_score = max(highest_score, cur_sim)
        if highest_score == cur_sim:
            bn_examples = synset['examples']
            matched_synset = {'glosses':bn_glosses,'examples':bn_examples}
            
    if matched_synset:
        for bn_gloss in matched_synset['glosses']:
            pd.DataFrame([[word, bn_gloss, 'bn_gloss']], columns=columns).to_sql(table_name, con=new_engine, if_exists='append', index=False)

        for bn_example in matched_synset['examples']:
            pd.DataFrame([[word, bn_example, 'bn_example']], columns=columns).to_sql(table_name, con=new_engine, if_exists='append', index=False)
    
#     print("DB gloss: {}".format(gloss))
#     print("Matched gloss: {}".format(matched_synset))
r_results.close()


aardvark
aardwolf
abandon
abbey
abdomen
abdominal
ability
able
abnormal
abnormality
abnormally
aboard
abound
about
above
abracadabra
abrasion
abroad
abrupt
abruptly
absence
absent
absolute
absolutely
absorb
absorbent
absorption
absorptive
abstinence
abstract
abstraction
abstractly
abundance
abundant
abuse
acacia
academic
academy
accelerate
acceleration
accelerator
accelerometer
accent
accept
acceptable
acceptance
acceptor
access
accessible
accessory
accident
accidental
accidentally
accommodate
accommodate
accompany
accomplish
accomplishment
accordance
accordingly
accordion
account
accountable
accumulate
accumulation
accuracy
accurate
accurately
acellular
acetyl
acetylene
ache
achieve
achievement
achromatic
acid
acidic
acidity
acknowledge
acknowledgement
acknowledgment
acne
acorn
acoustic
acquire
acre
acrobat
acrosome
across
act
actin
action
activate
activation
activator
active
actively
activist
activity
actor
actress
actual
actuality
actually
acute
ad
adapt
adaptation
adaptive
add
adda

blade
bladed
blame
blank
blanket
blare
blast
blasted
blastocyst
blaze
bleach
bleach
bleached
bleat
bleed
blend
blender
bless
blighter
blind
blindfold
blindfold
blindfolded
blindness
blink
bliss
blister
blistered
blithering
blizzard
blob
block
blockage
blog
bloke
blonde
blood
bloodroot
bloodstream
bloodwort
bloody
bloom
blossom
blossom
blot
blotch
blouse
blow
blow
blow
blowfly
blubber
blue
bluebell
blueberry
bluebird
bluebottle
bluefin
bluefish
bluff
bluff
bluff
bluff
bluish
blunder
blunt
blur
blurt
blush
blustery
boa
boar
board
boast
boat
boatman
bob
bobby
bobcat
bodied
bodily
body
body part
bog
boil
boiler
boisterous
bold
boldly
bole
Bolivia
Bolivian
bolt
bomb
bonanza
bond
bond
bond
bone
bonelike
bonfire
bong
bongo
bonk
bonnet
bonobo
bony
boo
booby
boogie
book
bookcase
bookend
booklet
boom
boomerang
boost
booster
boot
booth
booty
bop
borax
bore
bore
bore
boreal
bored
boron
borrow
Bosnia
boss
bossy
Boston
botanical
botanist
botany
bother
bother
bothered
bothersome
Botswana
bottle
bottl

commuter
compact
compact
compaction
compactness
companion
companionship
company
comparative
comparatively
compare
comparison
compartment
compass
compatible
compel
compensate
compete
competent
competition
competitive
compile
complain
complement
complementary
complete
completely
completion
complex
complexity
compliance
complicate
compliment
component
compose
composite
composition
compost
compound
comprehend
comprehensible
comprehension
comprehensive
compress
compression
comprise
compromise
compulsion
compulsive
compulsively
computation
compute
computer
con
conceal
conceive
concentrate
concentration
concentration
concept
conception
conceptual
concern
concert
concession
concise
concisely
conclude
conclusion
conclusively
concrete
condensation
condense
condition
conditioner
condom
conduct
conductance
conduction
conductivity
conductor
conduit
cone
coney
confer
conference
confess
confetti
confidence
confident
confidential
confine
confinement
confirm
conflict
conform
conformity
confuse
confusio

distract
distress
distribute
distribution
district
disturb
disturbance
disuse
ditch
ditty
diurnal
dive
diver
diverge
divergent
diverse
diversification
diversify
diversion
diversity
divide
divisible
division
divorce
divulge
dizzy
Djibouti
do
dobson
doc
dock
doctor
doctrine
document
documentation
dodder
doddle
dodge
dodo
dog
doggy
dogma
dole
doll
dollar
dollop
dolly
dolphin
domain
dome
domestic
domesticate
domestication
dominance
dominant
dominate
Dominica
Dominican
domino
don
donate
donkey
donut
doodle
doom
door
doorbell
doorknob
doormat
doorstep
doorway
dopey
dormancy
dormant
dormouse
dorsal
dose
dot
double
double-decker
doubloon
doubly
doubt
dough
doughnut
dove
down
down-and-out
downhill
downpour
downside
downstairs
downward
downwards
downy
doze
dozen
dozy
draft
drag
dragon
dragonfly
drain
drainage
drainpipe
drake
drama
dramatic
dramatically
dramatization
drape
drastic
drastically
draught
draw
drawback
drawer
drawing
dray
dread
dreadful
dreadfully
dream
dreamy
drench
dress
dribble
dri

forecast
forecaster
forehead
foreign
forelimb
forest
forestry
forever
forfeit
forget
forget-me-not
fork
form
formal
formaldehyde
formally
format
formation
formation
former
formerly
formula
formulate
forth
fortify
fortitude
fortnight
fortunately
fortune
forty
forty-five
forward
fossa
fossil
fossilization
fossilize
foster
foul
foundation
founded
founder
founding
fountain
four
fourteen
fourth
fowl
fox
foxglove
foxhound
foxy
foyer
fraction
fractional
fracture
fracture
fracture
fracture
fragile
fragment
fragment
fragment
fragmentation
fragrant
frame
framework
franc
France
frank
frantically
freckle
free
freedom
freely
freeman
freeway
freeze
freezer
freight
French
frequency
frequent
frequently
fresh
freshly
freshwater
Fresno
friar
friction
frictional
Friday
fridge
friend
friendliness
friendly
friendship
frigate
fright
frighten
frightfully
frill
fringe
frog
frogfish
frond
front
frost
frosted
frosty
frothy
frown
frown
fructose
fruit
fruiting
frustration
fry
ft
fucus
fuddle
fudge
fuel
fulcrum
fu

...

orchestra
orchid
order
orderly
ordinary
ordnance
ore
Oregon
organ
organelle
organic
organically
organise
organised
organism
organization
organizational
organize
organizer
orient
oriental
orientation
origin
original
originally
originate
oriole
ornament
ornate
ornithischian
ornithologist
orphan
orthographic
orthomyxovirus
oryx
os
oscillate
osmosis
osprey
ossification
ossify
osteoarthritis
osteoblast
osteoclast
osteocyte
osteoporosis
ostrich
other
otherwise
otherworldly
otter
ounce
out
outbreak
outcome
outdoor
outdoors
outer
outermost
outfit
outfit
outgoing
outgrowth
outlet
outline
outnumber
output
outright
outside
outstanding
outward
outwards
oval
oval
ovary
oven
over
overall
overbalance
overboard
overcast
overcome
overcrowd
overdose
overeat
overexpose
overflow
overgrow
overhang
overhang
overhead
overhear
overjoyed
overlap
overlap
overlie
overload
overload
overloaded
overnight
overpopulate
overpopulation
override
override
oversee
oversized
overtake
overturn
overview
overwhelmingly
ovipar

proximity
prune
pry
pseudopod
psychedelic
psychoactive
psychoanalysis
psychological
pub
puberty
pubic
public
publication
publish
pudding
puddle
Puebla
puff
puffball
puffin
pull
pulley
pullover
pulmonary
pulp
pulpy
pulse
pulverize
pumice
pump
pumpkin
pun
punch
punctual
punctuate
punish
pup
pupa
pupil
puppet
puppy
purchase
pure
purebred
purely
purification
purify
purple
purpose
purposefully
purr
purse
pursue
pursuit
push
pushchair
pussyfoot
put
puzzle
puzzler
pygmy
pyjamas
pylon
pyramid
pyrite
pyrrhotite
python
Q
Qatar
quack
quack-quack
quad
quadruped
quail
quake
qualification
qualitative
quality
quantifier
quantify
quantitative
quantity
quarrel
quarrelsome
quarry
quart
quarter
quartz
quaternary
queasy
Quebec
queen
queen
quest
question
questionable
questionnaire
queue
queue
quick
quickly
quicksand
quiet
quieten
quietly
quill
quilt
quit
quite
quiver
quiz
quota
r
rabbit
rabble
rabies
raccoon
race
rack
racket
racket
racquetball
radar
radial
radian
radiant
radiate
radiation
radical
radicle
r

shiny
ship
shipbuilding
shipwreck
shire
shirt
shiver
shivery
shock
shoe
shoelace
shoo
shoot
shoot
shoot
shoot
shooter
shop
shop
shop
shopkeeper
shopper
shore
shorebird
shoreline
short
short-lived
shortage
shortbread
shorten
shortly
shortness
shot
shot
shoulder
shout
shout
shout
shove
shovel
show
shower
showroom
showy
shred
shred
shrew
shrewd
shrewdly
shriek
shrill
shrimp
shrink
shrivel
shrub
shrug
shudder
shuffle
shuffle
shuffle
shush
shut
shut
shuttle
shuttle
shy
shyly
shyness
Siamese
Siberian
sibling
Sichuan
sick
sickle
sickness
side
siderite
sidewalk
sideways
sieve
sieve
sift
sigh
sigh
sight
sighting
sign
signal
signatory
significance
significant
significantly
signpost
sika
silence
silent
silently
silica
silicon
silk
silken
silky
silly
silt
silver
silverfish
silverware
silvery
similar
similarity
similarly
simmer
simple
simplex
simplify
simply
simulate
simulation
simultaneously
sincerely
sing
Singapore
singer
single
single
single
singular
sink
sinkhole
Sion
sip
sip
siphon
sir
sire
si

tableware
tabular
tach
tachometer
tackle
tackle
tackle
tactfully
tadpole
taffeta
tag
tail
tailgate
tailless
tailor
Tajikistan
take
takeover
takin
talapoin
talc
tale
talent
talk
talk
tall
tally
tally
talon
tamandua
tamarin
tame
tame
tan
tanager
tangible
tangle
tangle
tangled
tank
tank
tanker
tantalite
tantrum
Tanzania
tap
tape
tape
tape
tape
tape
taped
taped
taper
tapeworm
tapir
taproot
tarantula
target
tarnish
tarsier
tart
tartan
task
Tasmanian
taste
tasteless
taster
tasty
tattered
tattoo
tattoo
tawny
tax
taxi
taxon
taxonomic
taxonomy
tayra
tea
teach
teacher
teacup
teal
team
teapot
tear
tear
tear
tear
teardrop
tearful
tease
teaspoon
teaspoon
technical
technically
technician
technique
technological
technology
tectonic
teddy
tedious
tee
teem
teen
teenager
telecom
telecommunication
telephone
telescope
television
tell
teller
telly
telophase
temp
temper
temperate
temperature
tempered
template
temple
temporal
temporarily
temporary
tempt
ten
tenacity
tend
tendency
tender
tendon
tendril
tenia


wallpaper
walnut
walrus
waltz
wand
wander
wane
want
war
warbler
ward
warden
wardrobe
warm
warmth
warn
warning
warp
warrant
warren
warrior
wart
warty
wash
washbasin
washer
washing-up
Washington
wasp
waste
wastewater
watch
water
waters
waterbuck
watercress
waterfall
waterfowl
watermelon
waterproof
waterway
waterwheel
watery
watt
wave
wavelength
wavelike
wavy
wax
waxwork
waxy
way
weak
weaken
weakness
wealth
weapon
weaponry
wear
wearer
wearily
weary
weasel
weather
weatherman
weave
weaver
web
webbed
webbing
wed
wedding
wedge
Wednesday
weed
week
weekday
weekend
weekly
weep
weevil
weigh
weight
weightless
weightlessness
weightlifter
weir
weird
welcome
weld
well
well-kept
well-meaning
wellbeing
wellness
Welsh
west
westerly
western
westward
wet
wetland
wetness
whale
whatever
wheat
wheedle
wheel
wheelbarrow
wheelchair
wheeze
whelk
when
where
whereby
wherever
while
whimper
whine
whip
whip
whiplike
whiptail
whir
whirl
whirligig
whisk
whisk
whisker
whiskered
whisper
whistle
whit
white
whitish
whiz
w

In [3]:
from sqlalchemy import create_engine
import os
import pandas as pd
aligned_data_path = "aligned_data"
cleaned_filename = "FlatternedDB.db"
table_name = "t_vocab"
new_db_path = os.path.join(aligned_data_path, cleaned_filename)
new_engine = create_engine('sqlite:///{}'.format(new_db_path))

# new table columns to write
columns = ['Word','gloss','type']

df_data = pd.read_sql_table(table_name, con=new_engine)
df_data

Word  \
0                    aardvark   
1                    aardvark   
2                    aardvark   
3                    aardvark   
4                    aardvark   
5                    aardvark   
6                    aardvark   
7                    aardvark   
8                    aardvark   
9                    aardwolf   
10                   aardwolf   
11                   aardwolf   
12                   aardwolf   
13                   aardwolf   
14                   aardwolf   
15                   aardwolf   
16                   aardwolf   
17                    abandon   
18                    abandon   
19                    abandon   
20                    abandon   
21                    abandon   
22                    abandon   
23                    abandon   
24                      abbey   
25                      abbey   
26                      abbey   
27                      abbey   
28                      abbey   
29                      abbey   
...                       ...   
96338               zygospore   
96339               zygospore   
96340               zygospore   
96341                  zygote   
96342                  zygote   
96343                  zygote   
96344                  zygote   
96345                  zygote   
96346                  zygote   
96347                  zygote   
96348                  zygote   
96349       definite quantity   
96350       definite quantity   
96351       definite quantity   
96352     indefinite quantity   
96353     indefinite quantity   
96354     indefinite quantity   
96355  auditory communication   
96356  auditory communication   
96357  auditory communication   
96358        written language   
96359        written language   
96360        written language   
96361        measuring device   
96362        measuring device   
96363        measuring device   
96364        measuring device   
96365        measuring device   
96366        measuring device   
96367        measuring device   

                                                   gloss        type  
0                                               aardvark        self  
1      nocturnal burrowing mammal of the grasslands o...    db_gloss  
2      Nocturnal burrowing mammal of the grasslands o...    bn_gloss  
3      The aardvark is a medium-sized, burrowing, noc...    bn_gloss  
4                                    An animal in Africa    bn_gloss  
5                                      Species of mammal    bn_gloss  
6      A burrowing mammal (Orycteropus afer) of south...    bn_gloss  
7      The nocturnal, insectivorous, burrowing, mamma...    bn_gloss  
8      The aardvark burrows in the ground and feeds m...  bn_example  
9                                               aardwolf        self  
10     striped hyena of southeast Africa that feeds c...    db_gloss  
11     Striped hyena of southeast Africa that feeds c...    bn_gloss  
12     The aardwolf is a small, insectivorous mammal,...    bn_gloss  
13                                     Species of mammal    bn_gloss  
14     A carnivorous quadruped of the species Protele...    bn_gloss  
15     The nocturnal, insectivorous, mammal, Proteles...    bn_gloss  
16                The mammal species Proteles cristatus.    bn_gloss  
17                                               abandon        self  
18                                forsake, leave behind;    db_gloss  
19     We abandoned the old car in the empty parking lot  db_example  
20                                 Forsake, leave behind    bn_gloss  
21     To leave behind definitively or with serious c...    bn_gloss  
22     We abandoned the old car in the empty parking lot  bn_example  
23                                We abandoned the area.  bn_example  
24                                                 abbey        self  
25       a church associated with a monastery or convent    db_gloss  
26                    The convent of a community of nuns    b

In [13]:
# data enhancing
# 1. translation (DEL)
# Background text
# 2. concept in the same types are used as negative samples, others are random sampled
# randomly masking some words
# leave out a test set

## create full text index
# [refernce](https://appliedmachinelearning.blog/2018/07/31/developing-a-fast-indexing-and-full-text-search-engine-with-whoosh-a-pure-python-library/)
import os
from whoosh.index import create_in
from whoosh.fields import Schema, TEXT, ID
import shutil

import spacy
nlp = spacy.load('en_core_web_md')

def create_index(doc_path):
    schema = Schema(title=TEXT(stored=True),path=ID(stored=True),content=TEXT)
    
    if os.path.exists("indexdir"):
        shutil.rmtree('indexdir')
    if not os.path.exists("indexdir"):
        os.mkdir("indexdir")
 
    # Creating a index writer to add document as per schema
    ix = create_in("indexdir",schema)
    writer = ix.writer()
    with open(doc_path, "r") as in_file:
        for _idx, text in enumerate(in_file):
            norm_text = " ".join([token.lemma_.lower() for token in nlp(text) if token.is_alpha])
            writer.add_document(title="{}".format(text),\
              content=norm_text)
            if (_idx +1)% 10000 == 0:
                print('{} Finished'.format(_idx+1))
    writer.commit()
    print("Done.")

create_index("/home/chen/projects/text2concept/concept_space/data/Aristo-Mini-Corpus-Dec2016/Aristo-Mini-Corpus-Dec2016/Aristo-Mini-Corpus-Dec2016.txt")

10000 Finished
20000 Finished
30000 Finished
40000 Finished
50000 Finished
60000 Finished
70000 Finished
80000 Finished
90000 Finished
100000 Finished
110000 Finished
120000 Finished
130000 Finished
140000 Finished
150000 Finished
160000 Finished
170000 Finished
180000 Finished
190000 Finished
200000 Finished
210000 Finished
220000 Finished
230000 Finished
240000 Finished
250000 Finished
260000 Finished
270000 Finished
280000 Finished
290000 Finished
300000 Finished
310000 Finished
320000 Finished
330000 Finished
340000 Finished
350000 Finished
360000 Finished
370000 Finished
380000 Finished
390000 Finished
400000 Finished
410000 Finished
420000 Finished
430000 Finished
440000 Finished
450000 Finished
460000 Finished
470000 Finished
480000 Finished
490000 Finished
500000 Finished
510000 Finished
520000 Finished
530000 Finished
540000 Finished
550000 Finished
560000 Finished
570000 Finished
580000 Finished
590000 Finished
600000 Finished
610000 Finished
620000 Finished
630000 Finished
6

In [99]:
from whoosh.qparser import QueryParser
from whoosh import scoring
from whoosh.index import open_dir
import re

ix = open_dir("indexdir")
new_engine.execute("DROP TABLE IF EXISTS t_context")
t_context = "t_context"
with ix.searcher(weighting=scoring.Frequency) as searcher:
    for ith, word in enumerate(set(df_data.Word),1):
        try:
            lemma_w = " ".join([w.lemma_.lower() for w in nlp(word)])
            norm_w = " ".join([w.norm_.lower() for w in nlp(word)])
        except Exception as e:
            print(e)
            print("Error term {}th :{}".format(ith, word))
            continue
        query = QueryParser("content", ix.schema).parse(lemma_w)
        results = searcher.search(query,limit=5)
        cands = set()
        for res in results:
            valid_tokens = []
            anchor_idx = -1
            sentence = res['title']
            # remove noise data
            sr = re.search("#\{.+\}",sentence)
            if sr:
                sentence = sentence[sr.span()[1]:].strip()
            if not sentence: continue
                
            for token in nlp(sentence):
                if token.is_alpha:
                    valid_tokens.append(token.norm_)
                    
                    # keep the last one if duplicating
                    if token.lemma_ == lemma_w or lemma_w in token.lemma_ or token.lemma_ in lemma_w:
                        anchor_idx = len(valid_tokens) - 1
            cands.add(" ".join(valid_tokens[anchor_idx-1:anchor_idx+1]))
            cands.add(" ".join(valid_tokens[anchor_idx-1:anchor_idx+2]))
            cands.add(" ".join(valid_tokens[anchor_idx:anchor_idx+2]))
            cands.add(" ".join(valid_tokens[anchor_idx-2:anchor_idx+1]))
            cands.add(" ".join(valid_tokens[anchor_idx:anchor_idx+3])) 
        
        cands = {cand for cand in cands if cand}
        if word in cands:
            cands.remove(word)
        
        for w_context in cands:
            pd.DataFrame([[word, w_context, 'cp_context']], columns=columns).to_sql(t_context, con=new_engine, if_exists='append', index=False)
        if ith % 50 == 0: print("Finished {} terms.".format(ith) )
print("Done.")            

Finished 50 terms.
Finished 100 terms.
Finished 150 terms.
Finished 200 terms.
Finished 250 terms.
Finished 300 terms.
Finished 350 terms.
Finished 400 terms.
Finished 450 terms.
Finished 500 terms.
Finished 550 terms.
Finished 600 terms.
Finished 650 terms.
Finished 700 terms.
Finished 750 terms.
Finished 800 terms.
Finished 850 terms.
Finished 900 terms.
Finished 950 terms.
Finished 1000 terms.
Finished 1050 terms.
Finished 1100 terms.
Finished 1150 terms.
Finished 1200 terms.
Finished 1250 terms.
Finished 1300 terms.
Finished 1350 terms.
Finished 1400 terms.
Finished 1450 terms.
Finished 1500 terms.
Finished 1550 terms.
Finished 1600 terms.
Finished 1650 terms.
Finished 1700 terms.
Finished 1750 terms.
Finished 1800 terms.
Finished 1850 terms.
Finished 1900 terms.
Finished 1950 terms.
Finished 2000 terms.
Finished 2050 terms.
Finished 2100 terms.
Finished 2150 terms.
Finished 2200 terms.
Finished 2250 terms.
Finished 2300 terms.
Finished 2350 terms.
Finished 2400 terms.
Finished 245

In [100]:
new_engine.table_names()

['t_context', 't_vocab']

In [102]:
from sqlalchemy import create_engine
import os
import pandas as pd
aligned_data_path = "aligned_data"
cleaned_filename = "FlatternedDB.db"
table_name = "t_vocab"
t_context = "t_context"
new_db_path = os.path.join(aligned_data_path, cleaned_filename)
new_engine = create_engine('sqlite:///{}'.format(new_db_path))

# new table columns to write
columns = ['Word','gloss','type']

df_data = pd.read_sql_table(table_name, con=new_engine)
df_context = pd.read_sql_table(t_context, con=new_engine)
df = pd.concat([df_context,df_data], axis=0, ignore_index=True)
df

Word  \
0                         meow   
1                         meow   
2                         meow   
3                         meow   
4                         meow   
5                         meow   
6                         meow   
7                         meow   
8                         meow   
9                       breeze   
10                      breeze   
11                      breeze   
12                      breeze   
13                      breeze   
14                      breeze   
15                      breeze   
16                      breeze   
17                      breeze   
18                      breeze   
19                      breeze   
20                      breeze   
21                      breeze   
22                      breeze   
23                      breeze   
24                      breeze   
25                      breeze   
26                      breeze   
27                      breeze   
28                      breeze   
29                  reciprocal   
...                        ...   
267876               zygospore   
267877               zygospore   
267878               zygospore   
267879                  zygote   
267880                  zygote   
267881                  zygote   
267882                  zygote   
267883                  zygote   
267884                  zygote   
267885                  zygote   
267886                  zygote   
267887       definite quantity   
267888       definite quantity   
267889       definite quantity   
267890     indefinite quantity   
267891     indefinite quantity   
267892     indefinite quantity   
267893  auditory communication   
267894  auditory communication   
267895  auditory communication   
267896        written language   
267897        written language   
267898        written language   
267899        measuring device   
267900        measuring device   
267901        measuring device   
267902        measuring device   
267903        measuring device   
267904        measuring device   
267905        measuring device   

                                                    gloss        type  
0                                               a meow is  cp_context  
1                                         gonna cat meows  cp_context  
2                                               cat meows  cp_context  
3                                                 meow is  cp_context  
4                                            cat meows it  cp_context  
5                                             meow is the  cp_context  
6                                                  a meow  cp_context  
7                                                meows it  cp_context  
8                                          meows it makes  cp_context  
9                                             land breeze  cp_context  
10                                        the breezes are  cp_context  
11                                           breezes meet  cp_context  
12                                land breezes influenced  cp_context  
13                                            breezes are  cp_context  
14                                        and the breezes  cp_context  
15                                           land breezes  cp_context  
16                                       an inland breeze  cp_context  
17                                        breezes meet if  cp_context  
18                                      land breezes meet  cp_context  
19                                  breezes influenced by  cp_context  
20                                        and land breeze  cp_context  
21                                     breezes influenced  cp_context  
22                                          inland breeze  cp_context  
23                                     land breeze fronts  cp_context  
24                                            the breezes  cp_context  
25                                      breez

In [104]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
import os
import numpy as np
from gensim.models import Word2Vec

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


11/21/2018 11:02:36 - INFO - summarizer.preprocessing.cleaner -   'pattern' package not found; tag filters are not available for English


In [ ]:
class BidirectionalLSTM(nn.Module):
    """
    Bidirectional LSTM BaseLine
    """
    def __init__(self, input_size, num_layers, hidden_size, dim_concept):
        super(BidirectionalLSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dim_concept = dim_concept
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, dim_concept)  # 2 for bidirection
        
    def forward(self, features):
        # Set initial states
        h0, c0 = self.init_hidden(features)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out    
    
    def init_hidden(self, x):
        return (torch.randn(self.num_layers*2, x.size(0), self.hidden_size, device=device),
                torch.randn(self.num_layers*2, x.size(0), self.hidden_size, device=device))
        


In [ ]:
# fetch hidden features for the text
# find the concept vector in the knowledge graph

def random_mask(tokens):
    """
    20% tokens has 0.5 chance to be masked
    """
    if len(tokens) >= 5:
        cand_indices = np.random.choice(range(len(tokens)), len(tokens)//5, replace=False)
        for cand_idx in cand_indices:
            if np.random.random() > 0.5:
                tokens[cand_idx] = '[MASK]'
    return tokens

def get_bert_features(text, bert, tokenizer, transform=None)
    """
    Concat hidden states of the last 4 layers of the bert feature extractor
    """
    tokenized_text = tokenizer.tokenize(text)
    if transform:
        tokenized_text = transform(text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([0]*len(indexed_tokens))
    bert.eval()
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
    cat_last_4layers = torch.cat(encoded_layers[-4:], dim=-1).detach()
    return cat_last_4layers

class CorpusDataset(Dataset):
    """
    The dataset of the cleaned corpus
    """
    def __init__(self, df, pretrained_bert_name, concept_emb, transform=None):
        self.df = df
        self.transform = transform
        self.bert = BertModel.from_pretrained(pretrained_bert_name)
        self.tokenizer = BertTokenizer.from_pretrained("bert-large-uncased")
        self.concept_emb = concept_emb
        
    def __len__(self):
        return self.df.shape[0]
    
    
    
    def __getitem__(self, idx):
        sample = {}
        term, text, source = self.df.iloc[idx].values
        features = get_bert_features(text, self.bert, self.tokenizer, self.transform)
        sample['features'] = features        
        sample['concept_embedding'] = self.concept_emb[term]
        sample['type'] = source
        self.bert.eval()
        self.bert()
        
        return sample


    
dataset_loader = DataLoader(corpus_db, batch_size=8, shuffle=True, num_workers=4)

In [120]:
np.random.random()

0.5416887596827364

In [ ]:
class AverageMultipleLayers(nn.Module):
    """
    Average the last 4 BERT hidden layers
    """
    def __init__(self):
        super(AverageMultipleLayers, self).__init__()
        
    def forward(self, bert_hidden_layers):
        sentence_encoding = np.asarray(bert_hidden_layers[-4:]).mean(axis=0)

In [ ]:
class PoolingSingleLayer(nn.Module):
    """
    Plooling last but two
    """
    def __init__(self):
        super(PoolingSingleLayer, self).__init__()
        
    def forward(self, bert_hidden_layers):
        sentence_encoding = np.asarray

In [ ]:
class MultiCNNEncoder(nn.Module):
    """
    CNNInception Model
    """
    def __init__(self, opt):
        super(MultiCNNEncoder, self).__init__()
        sentence_convs = [nn.Sequential(
                nn.Conv1d(in_channels=opt.embedding_dim,
                          out_channels=opt.content_dim,
                          kernel_size=kernel_size),
                nn.BatchNorm1d(opt.content_dim),
                nn.ReLU(inplace=True),

                nn.Conv1d(in_channels=opt.content_dim,
                          out_channels=opt.content_dim,
                          kernel_size=kernel_size),
                nn.BatchNorm1d(opt.content_dim),
                nn.ReLU(inplace=True),
                nn.MaxPool1d(kernel_size=(opt.content_seq_len - kernel_size * 2 + 2))
            )
                for kernel_size in opt.kernel_sizes]

        self.sentence_convs = nn.ModuleList(sentence_convs)

        self.fc = nn.Sequential(
            nn.Linear(len(opt.kernel_sizes) * opt.content_dim * 2, opt.linear_hidden_size),
            nn.BatchNorm1d(opt.linear_hidden_size),
            nn.ReLU(inplace=True),
            nn.Linear(opt.linear_hidden_size, 1)
        )
    def forward(self, features):
        if self.opt.static:
            anchor_sent.detach()
            pos_sent.detach()

        sent_encodings = [sent_conv(features.permute(0, 2, 1)) for sent_conv in self.sentence_convs]
        encoding = torch.cat(sent_encodings, dim=1)
        # print(pos_pair.size())
        return self.fc((encoding.view(encoding.size(0), -1)))

        

In [ ]:
class DeepBiLSTM(nn.Module):
    """
    Deeper Bidirectional LSTM BaseLine
    """
    def __init__(self):
        super(DeepBiLSTM, self).__init__()
        
    def forward(self, bert_hidden_layers):
        pass

In [ ]:
rate = 0.2
if len(tokens) * rate > 1:
    m_cands = np.random.choice(range(len(tokens)), int(len(tokens)*rate))
    mask_indices = [i in m_cands if np.random.random() < 0.1]
    

In [ ]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
tokenized_text = "Who was Jim Henson ? Jim Henson was a puppeteer"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 6
tokenized_text[masked_index] = '[MASK]'
assert tokenized_text == ['who', 'was', 'jim', 'henson', '?', 'jim', '[MASK]', 'was', 'a', 'puppet', '##eer']

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])


model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# Predict hidden states features for each layer
encoded_layers, _ = model(tokens_tensor, segments_tensors)
assert len(encoded_layers) == 12`a

In [ ]:
# https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/02-intermediate/bidirectional_recurrent_neural_network/main.py

class BiRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size*2, num_classes)  # 2 for bidirection
    
    def forward(self, x):
        # Set initial states
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device) # 2 for bidirection 
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        
        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: tensor of shape (batch_size, seq_length, hidden_size*2)
        
        # Decode the hidden state of the last time step
        out = self.fc(out[:, -1, :])
        return out